In [4]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import re
from graphframes import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import *

In [5]:
sqlContext = SQLContext(sc)

In [22]:
path_to_graph='./centrality_computers.network'
def closeness(path_to_graph):
    with open(path_to_graph) as f:
        lines=f.readlines()
    #converting file to nodes and edges in rdd
    rdd_lines= sc.parallelize(lines)
    rdd_nodes = rdd_lines.flatMap(lambda x: re.split(' |,',x.strip())).distinct().map(lambda x: (x,))
    rdd_edges = rdd_lines.map(lambda x: tuple(re.split(' |,',x.strip())))
    #print rdd_edges.collect()
    #print rdd_nodes.collect()
    
    #converting rdd to dataframe
    df_nodes = sqlContext.createDataFrame(rdd_nodes, ['id'])
    df_edges = sqlContext.createDataFrame(rdd_edges, ['src', 'dst'])
    
    #Initializing graphframe
    graph = GraphFrame(df_nodes, df_edges)
    #graph.vertices.show()
    
    nodes = graph.vertices.flatMap(lambda x: x).collect()
    shortest_paths = graph.shortestPaths(landmarks=nodes)
    #shortest_paths.show()
    closeness_table = shortest_paths.select('id', explode('distances')).groupBy('id').agg((1.0/sum('value')).alias('Closeness Measure'))
    closeness_table.show()
    
closeness(path_to_graph)

+---+
| id|
+---+
|  A|
|  I|
|  E|
|  H|
|  D|
|  C|
|  G|
|  J|
|  B|
|  F|
+---+

+---+--------------------+
| id|   Closeness Measure|
+---+--------------------+
|  A| 0.05555555555555555|
|  B|0.058823529411764705|
|  C| 0.07142857142857142|
|  D| 0.06666666666666667|
|  E|0.058823529411764705|
|  F| 0.07142857142857142|
|  G| 0.05555555555555555|
|  H| 0.06666666666666667|
|  I|0.047619047619047616|
|  J|0.034482758620689655|
+---+--------------------+

